# Notebook for reproduce experiments of the paper doc2vote paper

In [1]:
from classes.dataset import Dataset
from classes.evaluator import Evaluator
from classes.model import BM25

import ir_datasets
import json

In [2]:
def extract_json(path):

	 with open(path) as f:

	 	d = json.load(f)
	 	return d

### Create the dataset instance and initialize the attributes of the reduced dataset

In [3]:
top_n=10


dataset_config = extract_json("/Users/marco/Documents/GitHub/ECAI-doc2vote/code/configuration/configDatasetMarco.json")

dataset=Dataset(dataset_config['dataset_name_test_2'])
#extarct all the passage of the dataset reduce
dataset.list_raduce_passage()
#add all the augmented file, it is applied a filter becouse it in not able
#to create a dictionary with the complete dataset
dataset.add_augmented_file()
#create the feature 
dataset.raduce_passage()
#join togheter the passage base and the passage augmented 
dataset.join_passage_base_aug(40)

Loading dataset shards:   0%|          | 0/55 [00:00<?, ?it/s]

### Test for classical test with model BM25 on the reduce dataset MSMarco

In [4]:
'''model = BM25(list(dataset.reduce_passages.values()))

score, dict_prediction = model.ranking_score(dataset.query, dataset.reduce_passages, top_n)

evaluator= Evaluator(dict_prediction,dataset.query_passage)

rank=evaluator.extract_ranking(top_n)

print("BM25", evaluator.mean_reciprocal_rank(list(rank.values())))'''

'model = BM25(list(dataset.reduce_passages.values()))\n\nscore, dict_prediction = model.ranking_score(dataset.query, dataset.reduce_passages, top_n)\n\nevaluator= Evaluator(dict_prediction,dataset.query_passage)\n\nrank=evaluator.extract_ranking(top_n)\n\nprint("BM25", evaluator.mean_reciprocal_rank(list(rank.values())))'

### Test for doc2vote test with model BM25 on the reduce dataset MSMarco 

In [5]:
model = BM25(list(dataset.reduce_passages_base_aug.values()))


score, dict_prediction = model.ranking_score(dataset.query, dataset.reduce_passages_base_aug, top_n)

evaluator= Evaluator(dict_prediction,dataset.query_passage)

rank=evaluator.extract_ranking(top_n)


print("BM25 with doc2vec:", evaluator.mean_reciprocal_rank(list(rank.values())))

BM25 with doc2vec: 0.9375732750469962


### Test for doc2vote test with model BM25 on the reduce dataset MSMarco indipendendent from position

In [6]:
'''top_n=10
aug_num=20
dataset.passage_creation_vote(aug_num)

list_passages=[x[1] for x in dataset.list_doc2vote]
index_passages=[x[0] for x in dataset.list_doc2vote]

model = BM25(list(list_passages))

score, dict_prediction = model.ranking_score_partial(dataset.query, list_passages, index_passages, top_n*aug_num)

from collections import Counter
counts_per_id = {
    key: dict(sorted(Counter(value).items(), key=lambda item: item[1], reverse=True))
    for key, value in dict_prediction.items()
}

dict_new={}
for j in counts_per_id.keys():
    
    dict_new[j]=counts_per_id[j].keys()

evaluator= Evaluator(dict_new,dataset.query_passage)

rank=evaluator.extract_ranking(10)

print("BM25 with doc2vote:", evaluator.mean_reciprocal_rank(list(rank.values())))'''

'top_n=10\naug_num=20\ndataset.passage_creation_vote(aug_num)\n\nlist_passages=[x[1] for x in dataset.list_doc2vote]\nindex_passages=[x[0] for x in dataset.list_doc2vote]\n\nmodel = BM25(list(list_passages))\n\nscore, dict_prediction = model.ranking_score_partial(dataset.query, list_passages, index_passages, top_n*aug_num)\n\nfrom collections import Counter\ncounts_per_id = {\n    key: dict(sorted(Counter(value).items(), key=lambda item: item[1], reverse=True))\n    for key, value in dict_prediction.items()\n}\n\ndict_new={}\nfor j in counts_per_id.keys():\n    \n    dict_new[j]=counts_per_id[j].keys()\n\nevaluator= Evaluator(dict_new,dataset.query_passage)\n\nrank=evaluator.extract_ranking(10)\n\nprint("BM25 with doc2vote:", evaluator.mean_reciprocal_rank(list(rank.values())))'

### Test for doc2vote test with model BM25 on the reduce dataset MSMarco dipendent from position

In [ ]:
from collections import Counter
import numpy as np

top_n = 10
aug_num = 40
dataset.passage_creation_vote(aug_num)

list_passages = [x[1] for x in dataset.list_doc2vote]
index_passages = [x[0] for x in dataset.list_doc2vote]

model = BM25(list(list_passages))

score, dict_prediction = model.ranking_score_partial(dataset.query, list_passages, index_passages, top_n * aug_num)

# Weighted vote counting
counts_per_id = {}

for key, value in dict_prediction.items():
    weighted_votes = {}
    
    for rank, passage_id in enumerate(value, start=1):  # Rank starts from 1
        weight = 1 / rank  # Reciprocal rank weighting (can be changed)
        weighted_votes[passage_id] = weighted_votes.get(passage_id, 0) + weight
    
    # Sort by weighted votes in descending order
    counts_per_id[key] = dict(sorted(weighted_votes.items(), key=lambda item: item[1], reverse=True))

# Extract ranked lists
dict_new = {j: list(counts_per_id[j].keys()) for j in counts_per_id.keys()}

evaluator = Evaluator(dict_new, dataset.query_passage)

rank = evaluator.extract_ranking(10)

print("BM25 with weighted doc2vote:", evaluator.mean_reciprocal_rank(list(rank.values())))


In [ ]:
from collections import Counter
import numpy as np

top_n = 10
aug_num = 40
dataset.passage_creation_vote(aug_num)

list_passages = [x[1] for x in dataset.list_doc2vote]
index_passages = [x[0] for x in dataset.list_doc2vote]

model = BM25(list(list_passages))

score, dict_prediction = model.ranking_score_partial(dataset.query, list_passages, index_passages, top_n * aug_num)

# Weighted vote counting
counts_per_id = {}

for key, value in dict_prediction.items():
    weighted_votes = {}
    
    for rank, passage_id in enumerate(value, start=1):  # Rank starts from 1
        weight = score[passage_id] # Reciprocal rank weighting (can be changed)
        weighted_votes[passage_id] = weighted_votes.get(passage_id, 0) + weight
    
    # Sort by weighted votes in descending order
    counts_per_id[key] = dict(sorted(weighted_votes.items(), key=lambda item: item[1], reverse=True))

# Extract ranked lists
dict_new = {j: list(counts_per_id[j].keys()) for j in counts_per_id.keys()}

evaluator = Evaluator(dict_new, dataset.query_passage)

rank = evaluator.extract_ranking(10)

print("BM25 with weighted doc2vote:", evaluator.mean_reciprocal_rank(list(rank.values())))